In [77]:
import pandas as pd
import os
from IPython.display import display, Markdown, HTML
%matplotlib inline
import tkinter as tk
from tkinter import filedialog, messagebox
from openpyxl import load_workbook
root = tk.Tk()
root.withdraw()
root.attributes("-topmost", True)

''

In [78]:
messagebox.showinfo('OnSSET', 'Open the input file with calibrated GIS data')
input_file = filedialog.askopenfilename()
df = pd.read_csv(input_file)

In [79]:
df.sort_values(by=['Pop2018'], ascending=False, inplace=True)
cumulative_pop = df['Pop2018'].cumsum()/df['Pop2018'].sum()

First, update the population and urbanization rate

In [80]:
years = [2025, 2030, 2040, 2050, 2060, 2070]
time_step = {2025: 7,
             2030: 5,
             2040: 10,
             2050: 10,
             2060: 10,
             2070: 10}

In [81]:
urbanization_rate = {2018: 0.2006, # Do not change the 2018 value, update the rest
                    2025: 0.263,
                    2030: 0.300,
                    2040: 0.375,
                    2050: 0.450,
                    2060: 0.525,
                    2070: 0.600}

population = {2018: 107535000, # Do not change the 2018 value, update the rest
              2025: 134208459,
              2030: 149774797,
              2040: 181383802,
              2050: 211590946,
              2060: 237724184,
              2070: 261168812}

household_size = {2018: 4.53, # Do not change the 2018 value, update the rest
                  2025: 4.26,
                  2030: 4.07,
                  2040: 3.69,
                  2050: 3.30,
                  2060: 3.00,
                  2070: 2.96}

In [82]:
for year in years:
    urban_growth = (urbanization_rate[year] * population[year])/(urbanization_rate[year-time_step[year]] * population[year-time_step[year]])
    rural_growth = ((1-urbanization_rate[year]) * population[year])/((1-urbanization_rate[year-time_step[year]]) * population[year-time_step[year]])
    df.loc[df['IsUrban'] == 2, 'Pop{}'.format(year)] = df['Pop{}'.format(year-time_step[year])] * urban_growth
    df.loc[df['IsUrban'] < 2, 'Pop{}'.format(year)] = df['Pop{}'.format(year-time_step[year])] * rural_growth
    df['NumPeoplePerHH' + '{}'.format(year)] = household_size[year] 

In [83]:
tier_1_urban = 19.1  # kWh/household/year
tier_2_urban = 229
tier_3_urban = 619
tier_4_urban = 1915
tier_5_urban = 3785

tier_1_rural = 5.9  # kWh/household/year
tier_2_rural = 181
tier_3_rural = 572
tier_4_rural = 1867
tier_5_rural = 3737

In [84]:
tier_split_2025 = {5: 0.03, # Share of population in Tier 5 
                   4: 0.14, # Share of population in Tier 5 or Tier 4
                   3: 0.14, # Share of population in Tier 5 or Tier 4 or Tier 3
                   2: 0.29, # Share of population in Tier 5 or Tier 4 or Tier 2
                   1: 1} # Share of population in Tier 5 or Tier 4 or Tier 2 or Tier 1 (Always = 1)

In [85]:
tier_split_2030 = {5: 0.03,
                   4: 0.20,
                   3: 0.21,
                   2: 0.37,
                   1: 1}

In [86]:
tier_split_2040 = {5: 0.05,
                   4: 0.38,
                   3: 0.45,
                   2: 0.63,
                   1: 1}

In [87]:
tier_split_2050 = {5: 0.07,
                   4: 0.49,
                   3: 0.90,
                   2: 1,
                   1: 1}

In [90]:
tier_split_2060 = {5: 0.11,
                   4: 0.59,
                   3: 1,
                   2: 1,
                   1: 1}

In [91]:
tier_split_2070 = {5: 0.16,
                   4: 0.67,
                   3: 1,
                   2: 1,
                   1: 1}

In [92]:
tier_splits = {2025: tier_split_2025,
               2030: tier_split_2030,
               2040: tier_split_2040,
               2050: tier_split_2050,
               2060: tier_split_2060,
               2070: tier_split_2070}

In [93]:
for year in years:
    tier_split = tier_splits[year]
    
    # Tier 1
    df.loc[df['IsUrban'] == 2, 'ResidentialDemandTierCustom' + "{}".format(year)] = tier_1_urban / household_size[year]
    df.loc[df['IsUrban'] < 2, 'ResidentialDemandTierCustom' + "{}".format(year)] = tier_1_rural / household_size[year]
    df['Tier' + "{}".format(year)] = 1
    # Tier 2
    df.loc[(df['IsUrban'] == 2) & (cumulative_pop <= tier_split[2]), 'ResidentialDemandTierCustom' + "{}".format(year)] = tier_2_urban / household_size[year]
    df.loc[(df['IsUrban'] < 2) & (cumulative_pop <= tier_split[2]), 'ResidentialDemandTierCustom' + "{}".format(year)] = tier_2_rural / household_size[year]
    df.loc[cumulative_pop <= tier_split[2], 'Tier' + "{}".format(year)] = 2
    # Tier 3
    df.loc[(df['IsUrban'] == 2) & (cumulative_pop <= tier_split[3]), 'ResidentialDemandTierCustom' + "{}".format(year)] = tier_3_urban / household_size[year]
    df.loc[(df['IsUrban'] < 2) & (cumulative_pop <= tier_split[3]), 'ResidentialDemandTierCustom' + "{}".format(year)] = tier_3_rural / household_size[year]
    df.loc[cumulative_pop <= tier_split[3], 'Tier' + "{}".format(year)] = 3
    # Tier 4
    df.loc[(df['IsUrban'] == 2) & (cumulative_pop <= tier_split[4]), 'ResidentialDemandTierCustom' + "{}".format(year)] = tier_4_urban / household_size[year]
    df.loc[(df['IsUrban'] < 2) & (cumulative_pop <= tier_split[4]), 'ResidentialDemandTierCustom' + "{}".format(year)] = tier_4_rural / household_size[year]
    df.loc[cumulative_pop <= tier_split[4], 'Tier' + "{}".format(year)] = 4
    # Tier 5
    df.loc[(df['IsUrban'] == 2) & (cumulative_pop <= tier_split[5]), 'ResidentialDemandTierCustom' + "{}".format(year)] = tier_5_urban / household_size[year]
    df.loc[(df['IsUrban'] < 2) & (cumulative_pop <= tier_split[5]), 'ResidentialDemandTierCustom' + "{}".format(year)] = tier_5_rural / household_size[year]
    df.loc[cumulative_pop <= tier_split[5], 'Tier' + "{}".format(year)] = 5

In [94]:
scenario_name = 'BAU_Updated_Test'

In [95]:
messagebox.showinfo('OnSSET', 'Browse to the folder where you want to save the outputs')
df.sort_index(inplace=True)
output_dir = filedialog.askdirectory()
output_dir_results = os.path.join(output_dir, '{}.csv'.format(scenario_name))
df.to_csv(output_dir_results, index=False)